In [42]:
class QuantumGate:
    """Représente une porte quantique dans un circuit."""
    def __init__(self, gate_type, targets, params=None):
        self.gate_type = gate_type  # Type de la porte : 'Rz', 'CNOT'
        self.targets = targets  # Liste des qubits cibles
        self.params = params  # Paramètres, ex : l'angle de rotation pour Rz

    def __repr__(self):
        return f"QuantumGate(type={self.gate_type}, targets={self.targets}, params={self.params})"

def phase_polynomial_from_circuit(circuit):
    """
    Convertit un circuit composé de portes Rz, CNOT, et NOT en polynôme de phase.
    
    :param circuit: Liste des portes dans le circuit.
    :return: Un polynôme de phase représentant le circuit.
    """
    # Phase polynomiale pour chaque qubit
    phase_poly = {}
    
    # Initialiser les polynômes pour chaque qubit
    for gate in circuit:
        for qubit in gate.targets:
            if qubit not in phase_poly:
                phase_poly[qubit] = []
    
    # Parcourir les portes du circuit pour construire le polynôme
    for gate in circuit:
        if gate.gate_type == 'Rz':
            qubit = gate.targets[0]
            angle = gate.params  # Paramètre d'angle pour la porte Rz
            phase_poly[qubit].append(angle)
        
        elif gate.gate_type == 'CNOT':
            control_qubit = gate.targets[0]
            target_qubit = gate.targets[1]
            # Modifier la dépendance du qubit cible par rapport au qubit de contrôle
            phase_poly[target_qubit].append(f"(x{control_qubit} ⊕ x{target_qubit})")
        
    return phase_poly

                
def commutation_cnot_rz(equation):
    if len(equation) < 2 :
        return equation 
    j = 0
    term = equation[j]
    if (isinstance(term, str) and'⊕' in term):
        return equation
    
    new_place = 0
    cnto_dict = {}
    while (j < len(equation)):
        good = True
        term = equation[j]
                
        if (isinstance(term, str) and'⊕' in term):
            if term in cnto_dict:
                cnto_dict[term] = cnto_dict[term] + 1
            else:
                cnto_dict[term] = 1
        
        for cnot , number in cnto_dict.items():
            if number % 2 != 0:
                good = False

        j = j+1
        if (good == True and len(cnto_dict.items()) != 0):
            new_place = j

    new_equation = []
    if (new_place != 0):
        rz_swap = equation[0]
        for i in range(1,new_place):
            new_equation.append(equation[i])
        new_equation.append(rz_swap)
        for i in range(new_place,len(equation)):
            new_equation.append(equation[i])
        return new_equation
    else:    
        return equation
        
        
        
def optimize_phase_polynomial(phase_polynomial):
    optimized_phase_poly = {}

    # Parcourir chaque qubit et son équation
    for qubit, equation in phase_polynomial.items():
        swap_equation = []
        while len(equation) != 0:
            first_term =  equation[0]
            equation = commutation_cnot_rz(equation)
            if(first_term == equation[0]):
                equation.remove(first_term)
                swap_equation.append(first_term)
        
        # Assigner la version optimisée à ce qubit
        optimized_phase_poly[qubit] = swap_equation
    
    return optimized_phase_poly

# Exemple de circuit avec des portes CNOT, Rz, et NOT
circuit = [
    QuantumGate("Rz", [1], 1),    
    QuantumGate("Rz", [1], 3),    
    QuantumGate("CNOT", [0, 1]),     
    QuantumGate("Rz", [1], 2),    
    QuantumGate("CNOT", [2, 1]),     
    QuantumGate("Rz", [1], 2.5),   
    QuantumGate("CNOT", [2, 1]),    
    QuantumGate("CNOT", [0, 1]),     
    QuantumGate("Rz", [1], 3),  
    QuantumGate("CNOT", [0, 1]),
]

def circuit_from_phase_polynomial(phase_poly):
    """
    Recrée un circuit à partir d'un polynôme de phase.
    
    :param phase_poly: Un dictionnaire représentant le polynôme de phase pour chaque qubit.
    :return: Une liste de portes quantiques qui correspond au polynôme.
    """
    circuit = []
    
    # Parcourir chaque qubit et ses termes dans le polynôme de phase
    for qubit, terms in phase_poly.items():
        for term in terms:
            if isinstance(term, str) and '⊕' in term:
                # C'est une porte CNOT
                control_qubit = int(term[term.find('x')+1:term.find('⊕')].strip())
                target_qubit = qubit
                cnot_gate = QuantumGate(gate_type='CNOT', targets=[control_qubit, target_qubit])
                circuit.append(cnot_gate)
            else:
                # C'est une porte Rz avec un certain angle
                rz_gate = QuantumGate(gate_type='Rz', targets=[qubit], params=term)
                circuit.append(rz_gate)
    
    return circuit


# Calculer le polynôme de phase pour ce circuit
polynomial = phase_polynomial_from_circuit(circuit)


print("Polynôme de phase pour chaque qubit:")


for qubit, terms in polynomial.items():
    print(f"Qubit {qubit}: {' , '.join(map(str, terms))}")
    
circuit_2 = circuit_from_phase_polynomial(polynomial)

print(circuit_2)
    
polynomial = optimize_phase_polynomial(polynomial)



# Afficher le polynôme de phase
print("Polynôme de phase pour chaque qubit:")
for qubit, terms in polynomial.items():
    print(f"Qubit {qubit}: {' , '.join(map(str, terms))}")




Polynôme de phase pour chaque qubit:
Qubit 1: 1 , 3 , (x0 ⊕ x1) , 2 , (x2 ⊕ x1) , 2.5 , (x2 ⊕ x1) , (x0 ⊕ x1) , 3 , (x0 ⊕ x1)
Qubit 0: 
Qubit 2: 
[QuantumGate(type=Rz, targets=[1], params=1), QuantumGate(type=Rz, targets=[1], params=3), QuantumGate(type=CNOT, targets=[0, 1], params=None), QuantumGate(type=Rz, targets=[1], params=2), QuantumGate(type=CNOT, targets=[2, 1], params=None), QuantumGate(type=Rz, targets=[1], params=2.5), QuantumGate(type=CNOT, targets=[2, 1], params=None), QuantumGate(type=CNOT, targets=[0, 1], params=None), QuantumGate(type=Rz, targets=[1], params=3), QuantumGate(type=CNOT, targets=[0, 1], params=None)]
Polynôme de phase pour chaque qubit:
Qubit 1: (x0 ⊕ x1) , (x2 ⊕ x1) , 2.5 , (x2 ⊕ x1) , (x0 ⊕ x1) , 3 , 1 , 3 , (x0 ⊕ x1) , 2
Qubit 0: 
Qubit 2: 


In [41]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
import numpy as np

# Définir le premier circuit basé sur le premier polynôme de phase
def create_circuit_1():
    qc = QuantumCircuit(3)
    
    # Ajouter les portes selon le polynôme de phase donné pour Qubit 1
    qc.rz(1, 1)  # Rz(1) sur qubit 1
    qc.rz(3, 1)  # Rz(3) sur qubit 1
    qc.cx(0, 1)  # CNOT entre qubit 0 et qubit 1
    qc.rz(2, 1)  # Rz(2) sur qubit 1
    qc.cx(2, 1)  # CNOT entre qubit 2 et qubit 1
    qc.rz(2.5, 1)  # Rz(2.5) sur qubit 1
    qc.cx(2, 1)  # CNOT entre qubit 2 et qubit 1
    qc.cx(0, 1)  # CNOT entre qubit 0 et qubit 1
    qc.rz(3, 1)  # Rz(3) sur qubit 1
    qc.cx(0, 1)  # CNOT entre qubit 0 et qubit 1
    
    return qc

# Définir le deuxième circuit basé sur le polynôme de phase optimisé
def create_circuit_2():
    qc = QuantumCircuit(3)
    
    # Ajouter les portes selon le polynôme de phase optimisé pour Qubit 1
    qc.cx(0, 1)  # CNOT entre qubit 0 et qubit 1
    qc.cx(2, 1)  # CNOT entre qubit 2 et qubit 1
    qc.rz(2.5, 1)  # Rz(2.5) sur qubit 1
    qc.cx(2, 1)  # CNOT entre qubit 2 et qubit 1
    qc.cx(0, 1)  # CNOT entre qubit 0 et qubit 1
    qc.rz(7, 1)  # Rz(3) sur qubit 1
    qc.cx(0, 1)  # CNOT entre qubit 0 et qubit 1
    qc.rz(2, 1)  # Rz(2) sur qubit 1
    
    return qc

# Créer les deux circuits
circuit_1 = create_circuit_1()
circuit_2 = create_circuit_2()
print(circuit_1)
print(circuit_2)
# Calculer les matrices unitaires des deux circuits
unitary_1 = Operator(circuit_1).data
unitary_2 = Operator(circuit_2).data

# Vérifier si les deux matrices sont équivalentes (à une phase globale près)
if np.allclose(unitary_1, unitary_2, atol=1e-10):
    print("Les deux circuits sont équivalents.")
else:
    print("Les deux circuits ne sont pas équivalents.")

                                                                             
q_0: ────────────────────■──────────────────────────────────■─────────────■──
     ┌───────┐┌───────┐┌─┴─┐┌───────┐┌───┐┌─────────┐┌───┐┌─┴─┐┌───────┐┌─┴─┐
q_1: ┤ Rz(1) ├┤ Rz(3) ├┤ X ├┤ Rz(2) ├┤ X ├┤ Rz(2.5) ├┤ X ├┤ X ├┤ Rz(3) ├┤ X ├
     └───────┘└───────┘└───┘└───────┘└─┬─┘└─────────┘└─┬─┘└───┘└───────┘└───┘
q_2: ──────────────────────────────────■───────────────■─────────────────────
                                                                             
                                                           
q_0: ──■─────────────────────────■─────────────■───────────
     ┌─┴─┐┌───┐┌─────────┐┌───┐┌─┴─┐┌───────┐┌─┴─┐┌───────┐
q_1: ┤ X ├┤ X ├┤ Rz(2.5) ├┤ X ├┤ X ├┤ Rz(7) ├┤ X ├┤ Rz(2) ├
     └───┘└─┬─┘└─────────┘└─┬─┘└───┘└───────┘└───┘└───────┘
q_2: ───────■───────────────■──────────────────────────────
                                                           
Les deux circuits sont équivalents

In [35]:
# Exemple de circuit avec des portes CNOT, Rz, et NOT
circuit = [
    QuantumGate("Rz", [1], "θ1"),    # Rz(θ1) sur qubit 1
    QuantumGate("CNOT", [0, 1]),     # CNOT entre qubit 0 (contrôle) et qubit 1 (cible) 
    QuantumGate("Rz", [1], "θ2"),    # Rz(θ2) sur qubit 
    QuantumGate("CNOT", [0, 1]),     # CNOT entre qubit 0 (contrôle) et qubit 1 (cible)
    QuantumGate("Rz", [0], "θ3"),    # Rz(θ3) sur qubit 1
    QuantumGate("Rz", [1], "θ4"),    # Rz(θ4) sur qubit 1
    QuantumGate("CNOT", [1, 0]),     # CNOT entre qubit 0 (contrôle) et qubit 1 (cible)
]

# Calculer le polynôme de phase pour ce circuit
polynomial = phase_polynomial_from_circuit(circuit)
print("Polynôme de phase pour chaque qubit:")
for qubit, terms in polynomial.items():
    print(f"Qubit {qubit}: {' , '.join(map(str, terms))}")
polynomial = optimize_phase_polynomial(polynomial)
# Afficher le polynôme de phase
print("Polynôme de phase pour chaque qubit:")
for qubit, terms in polynomial.items():
    print(f"Qubit {qubit}: {' , '.join(map(str, terms))}")

Polynôme de phase pour chaque qubit:
Qubit 1: θ1 , (x0 ⊕ x1) , θ2 , (x0 ⊕ x1) , θ4
Qubit 0: θ3 , (x1 ⊕ x0)
Polynôme de phase pour chaque qubit:
Qubit 1: (x0 ⊕ x1) , θ2 , (x0 ⊕ x1) , θ4 , θ1
Qubit 0: θ3 , (x1 ⊕ x0)


In [40]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
import numpy as np

# Définir le premier circuit basé sur le premier polynôme de phase
def create_circuit_1():
    qc = QuantumCircuit(2)
    
    # Qubit 1
    qc.rz(1, 1)  # Rz(1) sur qubit 1
    qc.cx(0, 1)  # CNOT entre qubit 0 et qubit 1
    qc.rz(2, 1)  # Rz(2) sur qubit 1
    qc.cx(0, 1)  # CNOT entre qubit 0 et qubit 1
    qc.rz(4, 1)  # Rz(4) sur qubit 1
    
    # Qubit 0
    qc.rz(3, 0)  # Rz(3) sur qubit 0
    qc.cx(1, 0)  # CNOT entre qubit 1 et qubit 0
    
    return qc

# Définir le deuxième circuit basé sur le deuxième polynôme de phase
def create_circuit_2():
    qc = QuantumCircuit(2)
    
    # Qubit 1
    qc.cx(0, 1)  # CNOT entre qubit 0 et qubit 1
    qc.rz(2, 1)  # Rz(2) sur qubit 1
    qc.cx(0, 1)  # CNOT entre qubit 0 et qubit 1
    qc.rz(4, 1)  # Rz(4) sur qubit 1
    qc.rz(1, 1)  # Rz(1) sur qubit 1
    
    # Qubit 0
    qc.rz(3, 0)  # Rz(3) sur qubit 0
    qc.cx(1, 0)  # CNOT entre qubit 1 et qubit 0
    
    return qc

# Créer les deux circuits
circuit_1 = create_circuit_1()
circuit_2 = create_circuit_2()
print(circuit_1)
print(circuit_2)
# Calculer les matrices unitaires des deux circuits
unitary_1 = Operator(circuit_1).data
unitary_2 = Operator(circuit_2).data

# Vérifier si les deux matrices sont équivalentes (à une phase globale près)
if np.allclose(unitary_1, unitary_2, atol=1e-10):
    print("Les deux circuits sont équivalents.")
else:
    print("Les deux circuits ne sont pas équivalents.")


                                 ┌───────┐┌───┐
q_0: ───────────■─────────────■──┤ Rz(3) ├┤ X ├
     ┌───────┐┌─┴─┐┌───────┐┌─┴─┐├───────┤└─┬─┘
q_1: ┤ Rz(1) ├┤ X ├┤ Rz(2) ├┤ X ├┤ Rz(4) ├──■──
     └───────┘└───┘└───────┘└───┘└───────┘     
                        ┌───────┐         ┌───┐
q_0: ──■─────────────■──┤ Rz(3) ├─────────┤ X ├
     ┌─┴─┐┌───────┐┌─┴─┐├───────┤┌───────┐└─┬─┘
q_1: ┤ X ├┤ Rz(2) ├┤ X ├┤ Rz(4) ├┤ Rz(1) ├──■──
     └───┘└───────┘└───┘└───────┘└───────┘     
Les deux circuits sont équivalents.


In [37]:
circuit = [
    QuantumGate("CNOT", [2, 1]),     # CNOT entre qubit 0 (contrôle) et qubit 1 (cible) 
    QuantumGate("Rz", [3], 1),    # Rz(θ1) sur qubit 1
    QuantumGate("CNOT", [2, 3]),     # CNOT entre qubit 0 (contrôle) et qubit 1 (cible) 
    QuantumGate("Rz", [3], 2),    # Rz(θ2) sur qubit 
    QuantumGate("CNOT", [1, 3]),     # CNOT entre qubit 0 (contrôle) et qubit 1 (cible) 
    QuantumGate("Rz", [3], 3),    # Rz(θ3) sur qubit 1
    QuantumGate("CNOT", [1, 3]), 
    QuantumGate("CNOT", [0, 3]), 
    QuantumGate("Rz", [3], 4),    # Rz(θ4) sur qubit 1
    QuantumGate("CNOT", [2, 3]),     # CNOT entre qubit 0 (contrôle) et qubit 1 (cible)
    
    QuantumGate("CNOT", [1, 3]), 
    QuantumGate("Rz", [3], 5),
    QuantumGate("CNOT", [0, 3]), 
    QuantumGate("CNOT", [1, 3]), 
]

polynomial = phase_polynomial_from_circuit(circuit)
print("Polynôme de phase pour chaque qubit:")
for qubit, terms in polynomial.items():
    print(f"Qubit {qubit}: {' , '.join(map(str, terms))}")
polynomial = optimize_phase_polynomial(polynomial)
# Afficher le polynôme de phase
print("Polynôme de phase pour chaque qubit:")
for qubit, terms in polynomial.items():
    print(f"Qubit {qubit}: {' , '.join(map(str, terms))}")

Polynôme de phase pour chaque qubit:
Qubit 2: 
Qubit 1: (x2 ⊕ x1)
Qubit 3: 1 , (x2 ⊕ x3) , 2 , (x1 ⊕ x3) , 3 , (x1 ⊕ x3) , (x0 ⊕ x3) , 4 , (x2 ⊕ x3) , (x1 ⊕ x3) , 5 , (x0 ⊕ x3) , (x1 ⊕ x3)
Qubit 0: 
Polynôme de phase pour chaque qubit:
Qubit 2: 
Qubit 1: (x2 ⊕ x1)
Qubit 3: (x2 ⊕ x3) , (x1 ⊕ x3) , 3 , (x1 ⊕ x3) , 2 , (x0 ⊕ x3) , 4 , (x2 ⊕ x3) , (x1 ⊕ x3) , 5 , (x0 ⊕ x3) , (x1 ⊕ x3) , 1
Qubit 0: 


In [38]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
import numpy as np

# Création du premier circuit correspondant au premier polynôme de phase
def create_circuit_1():
    qc = QuantumCircuit(4)  # Qubits : 0, 1, 2, 3

    # Qubit 1
    qc.cx(2, 1)  # CNOT entre qubit 2 et qubit 1

    # Qubit 3
    qc.rz(1, 3)  # Rz(1) sur qubit 3
    qc.cx(2, 3)  # CNOT entre qubit 2 et qubit 3
    qc.rz(2, 3)  # Rz(2) sur qubit 3
    qc.cx(1, 3)  # CNOT entre qubit 1 et qubit 3
    qc.rz(3, 3)  # Rz(3) sur qubit 3
    qc.cx(1, 3)  # CNOT entre qubit 1 et qubit 3
    qc.cx(0, 3)  # CNOT entre qubit 0 et qubit 3
    qc.rz(4, 3)  # Rz(4) sur qubit 3
    qc.cx(2, 3)  # CNOT entre qubit 2 et qubit 3
    qc.cx(1, 3)  # CNOT entre qubit 1 et qubit 3
    qc.rz(5, 3)  # Rz(5) sur qubit 3
    qc.cx(0, 3)  # CNOT entre qubit 0 et qubit 3
    qc.cx(1, 3)  # CNOT entre qubit 1 et qubit 3

    return qc

# Création du deuxième circuit correspondant au deuxième polynôme de phase
def create_circuit_2():
    qc = QuantumCircuit(4)  # Qubits : 0, 1, 2, 3

    # Qubit 1
    qc.cx(2, 1)  # CNOT entre qubit 2 et qubit 1

    # Qubit 3
    qc.cx(2, 3)  # CNOT entre qubit 2 et qubit 3
    qc.cx(1, 3)  # CNOT entre qubit 1 et qubit 3
    qc.rz(3, 3)  # Rz(3) sur qubit 3
    qc.cx(1, 3)  # CNOT entre qubit 1 et qubit 3
    qc.rz(2, 3)  # Rz(2) sur qubit 3
    qc.cx(0, 3)  # CNOT entre qubit 0 et qubit 3
    qc.rz(4, 3)  # Rz(4) sur qubit 3
    qc.cx(2, 3)  # CNOT entre qubit 2 et qubit 3
    qc.cx(1, 3)  # CNOT entre qubit 1 et qubit 3
    qc.rz(5, 3)  # Rz(5) sur qubit 3
    qc.cx(0, 3)  # CNOT entre qubit 0 et qubit 3
    qc.cx(1, 3)  # CNOT entre qubit 1 et qubit 3
    qc.rz(1, 3)  # Rz(1) sur qubit 3

    return qc

# Créer les deux circuits
circuit_1 = create_circuit_1()
circuit_2 = create_circuit_2()

print(circuit_1)
print(circuit_2)

# Calculer les matrices unitaires des deux circuits
unitary_1 = Operator(circuit_1).data
unitary_2 = Operator(circuit_2).data

# Vérifier si les deux matrices sont équivalentes (à une phase globale près)
if np.allclose(unitary_1, unitary_2, atol=1e-10):
    print("Les deux circuits sont équivalents.")
else:
    print("Les deux circuits ne sont pas équivalents.")


                                                                       »
q_0: ────────────────────────────────────────────■─────────────────────»
       ┌───┐                                     │                     »
q_1: ──┤ X ├──────────────────■─────────────■────┼──────────────────■──»
       └─┬─┘                  │             │    │                  │  »
q_2: ────■──────■─────────────┼─────────────┼────┼─────────────■────┼──»
     ┌───────┐┌─┴─┐┌───────┐┌─┴─┐┌───────┐┌─┴─┐┌─┴─┐┌───────┐┌─┴─┐┌─┴─┐»
q_3: ┤ Rz(1) ├┤ X ├┤ Rz(2) ├┤ X ├┤ Rz(3) ├┤ X ├┤ X ├┤ Rz(4) ├┤ X ├┤ X ├»
     └───────┘└───┘└───────┘└───┘└───────┘└───┘└───┘└───────┘└───┘└───┘»
«                        
«q_0: ───────────■───────
«                │       
«q_1: ───────────┼────■──
«                │    │  
«q_2: ───────────┼────┼──
«     ┌───────┐┌─┴─┐┌─┴─┐
«q_3: ┤ Rz(5) ├┤ X ├┤ X ├
«     └───────┘└───┘└───┘
                                                                            »
q_0: ──────────────────────────

In [39]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
import numpy as np

# Définir le premier circuit basé sur le premier polynôme de phase
def create_circuit_1():
    qc = QuantumCircuit(2)
    
    # Ajouter les portes selon le polynôme de phase donné pour Qubit 1
    qc.cx(0, 1)
    qc.rz(2, 1) 
    qc.cx(0, 1)
    qc.rz(5, 1) 
    qc.rz(3, 0)  
    qc.cx(1, 0)
    
    return qc

# Définir le deuxième circuit basé sur le polynôme de phase optimisé
def create_circuit_2():
    qc = QuantumCircuit(2)
    qc.rz(3, 0)  
    qc.rz(5, 1)  
    qc.cx(1, 0)
    qc.rz(2, 0)
    
    return qc

print(circuit_1)
print(circuit_2)

# Créer les deux circuits
circuit_1 = create_circuit_1()
circuit_2 = create_circuit_2()

# Calculer les matrices unitaires des deux circuits
unitary_1 = Operator(circuit_1).data
unitary_2 = Operator(circuit_2).data

# Vérifier si les deux matrices sont équivalentes (à une phase globale près)
if np.allclose(unitary_1, unitary_2, atol=1e-10):
    print("Les deux circuits sont équivalents.")
else:
    print("Les deux circuits ne sont pas équivalents.")

                                                                       »
q_0: ────────────────────────────────────────────■─────────────────────»
       ┌───┐                                     │                     »
q_1: ──┤ X ├──────────────────■─────────────■────┼──────────────────■──»
       └─┬─┘                  │             │    │                  │  »
q_2: ────■──────■─────────────┼─────────────┼────┼─────────────■────┼──»
     ┌───────┐┌─┴─┐┌───────┐┌─┴─┐┌───────┐┌─┴─┐┌─┴─┐┌───────┐┌─┴─┐┌─┴─┐»
q_3: ┤ Rz(1) ├┤ X ├┤ Rz(2) ├┤ X ├┤ Rz(3) ├┤ X ├┤ X ├┤ Rz(4) ├┤ X ├┤ X ├»
     └───────┘└───┘└───────┘└───┘└───────┘└───┘└───┘└───────┘└───┘└───┘»
«                        
«q_0: ───────────■───────
«                │       
«q_1: ───────────┼────■──
«                │    │  
«q_2: ───────────┼────┼──
«     ┌───────┐┌─┴─┐┌─┴─┐
«q_3: ┤ Rz(5) ├┤ X ├┤ X ├
«     └───────┘└───┘└───┘
                                                                            »
q_0: ──────────────────────────